## Foundations of Computer Science Project 

Professor: Gianluca Della Vedova

Project made by Liborio Adriano Mastrolia

Badge Number: 901970

You have to work on the [Dogs adoptions](https://drive.google.com/file/d/1wQsA0oB6wwYlnkvvcyBCmLk7QmgVWNax/view?usp=sharing) dataset. 

It contains three files:
*  `dogs.csv`, shortly *dogs*
*  `dogTravel.csv`, shortly *travels*
*  `NST-EST2021-POP.csv`

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).
1.    At most 3 students can be in each group. You must create the groups by yourself.
1.    You do not have to send me the project *before* the discussion.

First of all, we need to import Python libraries:

In [ ]:
import pandas as pd
import numpy as np
import difflib 

After, to visualize data better, we need to set some options:

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

We first need to read the files `dogs.csv`, `dogTravel.csv` and `NST-EST2021-POP.csv` and from the dogs adoptions dataset.

In [ ]:
dogs = pd.read_csv(r"C:\Users\mastr\OneDrive\Desktop\Università\Foundations of Computer Science\Project\Dataset\dogs.csv")

dogTravel = pd.read_csv(r"C:\Users\mastr\OneDrive\Desktop\Università\Foundations of Computer Science\Project\Dataset\dogTravel.csv")

Population = pd.read_csv(r"C:\Users\mastr\OneDrive\Desktop\Università\Foundations of Computer Science\Project\Dataset\NST-EST2021-POP.csv", names=['State','Population'])

We put 'r' before string due to the following error: 

- SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape

In this way, the path gets converted to a raw string from a normal string and can be read without problems. Moreover, since the last csv didn't have any header row, we add one directly in Python, without any need to alter the original file.

As a reference, we will now count the total amount of rows and columns on each of the three datasets, in order to understand the dimension of each one:

In [ ]:
dogs.shape

In [ ]:
dogTravel.shape

In [ ]:
Population.shape

We can now read each dataset, with the command 'head' which will show the first 5 rows of the dataset:

In [ ]:
dogs.head()

In [ ]:
dogTravel.head()

In [ ]:
Population.head()

### 1. Extract all dogs with status that is not *adoptable*

In [ ]:
not_adoptable_dogs = dogs[dogs['status'] != 'adoptable']

not_adoptable_dogs

### 2. For each (primary) breed, determine the number of dogs

Before proceeding, we need to check if there is any missing values (NaN):

In [ ]:
dogs['breed_primary'].isnull().values.any()

Since there is no 'Nan' values, we can determine the number of dogs for each primary breed

In [ ]:
primary_breed_dogs = dogs.groupby('breed_primary').count()['id']

primary_breed_dogs

Since the 'count' method counts the number of non-missing rows for each column, it is better to choose a column: that's why we referred to column 'id'

### 3. For each (primary) breed, determine the ratio between the number of dogs of `Mixed Breed` and those not of Mixed Breed. Hint: look at the `secondary_breed`.

In [ ]:
grouped_dogs = dogs.groupby('breed_primary')

mixed_breed_count = grouped_dogs['breed_secondary'].apply(lambda x: (x == 'Mixed Breed').sum())
not_mixed_count = grouped_dogs.size() - mixed_breed_count

ratio = mixed_breed_count / not_mixed_count

result_mixed_breed = pd.DataFrame({
    'Primary Breed': grouped_dogs.groups.keys(),
    'Mixed Breed Count': mixed_breed_count,
    'Not Mixed Breed Count': not_mixed_count,
    'Ratio': round(ratio, 3)
}).set_index('Primary Breed')

result_mixed_breed


### 4. For each (primary) breed, determine the earliest and the latest `posted` timestamp.

Moreover, in the column 'posted' there are a few values which are not datetime, but city names; we need to remove these values by dropping them:

In [ ]:
dogs['posted'] = pd.to_datetime(dogs['posted'], errors = 'coerce')

dogs = dogs.dropna(subset=['posted'])

Setting errors = 'coerce' means that invalid parsing will be set as NaT (Not a Time), and then it will be dropped with 'dropna' method. This will always be useful for task number 6. We can now move on

In [ ]:
earliest = dogs.groupby('breed_primary')['posted'].min()
latest = dogs.groupby('breed_primary')['posted'].max()

timestamp = pd.DataFrame({
    'Primary Breed': grouped_dogs.groups.keys(),
    'Earliest Timestamp': earliest,
    'Latest Timestamp': latest
}).set_index('Primary Breed')

timestamp

### 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

First of all, let's check for unique values on 'contact_state', since it will be the column we will use in the groupby operator:

In [ ]:
dogs['contact_state']

We can then perform task 5, since all values can be considered valid:

In [ ]:
grouped_states = dogs.groupby('contact_state')

sex_imbalance = grouped_states['sex'].apply(lambda x: (x == 'Male').sum() - (x == 'Female').sum())

state_with_largest_imbalance = sex_imbalance.idxmax()

result_sex_imbalance = pd.DataFrame({
    'State': grouped_states.groups.keys(),
    'Sex Imbalance': sex_imbalance
}).set_index('State')

result_sex_imbalance

After displaying all the data regarding the sex imbalance, we can find the state with the largest sex imbalance

In [ ]:
print("State with largest sex imbalance is", state_with_largest_imbalance, "with a sex imbalance of", result_sex_imbalance['Sex Imbalance'].max())

### 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [ ]:
grouped_age_size = dogs.groupby(['age', 'size'])

avg_duration = grouped_age_size['stay_duration'].mean()
avg_cost = grouped_age_size['stay_cost'].mean()

result_age_size = pd.DataFrame({
    'Age': [pair[0] for pair in grouped_age_size.groups.keys()],
    'Size': [pair[1] for pair in grouped_age_size.groups.keys()],
    'Average Duration of Stay': avg_duration,
    'Average Cost of Stay': avg_cost
}).set_index(['Age', 'Size'])

result_age_size

### 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [ ]:
dogs_travels = pd.merge(dogs, dogTravel, on = 'id', how='inner')

dogs_with_3_or_more_travels = dogs_travels.groupby('id').filter(lambda x: len(x) >= 3)

dogs_with_3_or_more_travels

In [ ]:
breeds_of_dogs_with_3_or_more_travels = dogs_with_3_or_more_travels['breed_primary'].unique()

breeds_of_dogs_with_3_or_more_travels.tolist()

### 8. Fix the `travels` table so that the correct state is computed from  the `manual` and the `found` fields. If `manual` is not missing, then it overrides what is stored in `found`.

In [ ]:
dogTravel['state'] = dogTravel.apply(lambda row: row['manual'] if pd.notnull(row['manual']) else row['found'], axis=1)

dogTravel.drop(['found', 'manual'], axis=1, inplace=True)

dogTravel

### 9. For each state, compute the ratio between the number of travels and the population.

In [ ]:
travels_by_state = dogTravel.groupby('state').size().reset_index(name='Travels')

state_population = pd.merge(travels_by_state, Population, left_on='state', right_on='State', how='inner')

state_population['Ratio'] = state_population['Travels'] / state_population['Population'].str.replace('.', '').astype(int)

state_population

### 10. For each dog, compute the number of days from the `posted` day to the day of last access.

In [ ]:
dogs['posted'] = pd.to_datetime(dogs['posted'], format='ISO8601')

dogs['accessed'] = pd.to_datetime(dogs['accessed']).dt.tz_localize('UTC')

dogs['days_since_posted'] = (dogs['accessed'] - dogs['posted']).dt.days

dogs_dates_days = pd.DataFrame({
    'id': dogs['id'].keys(),
    'posted': dogs['posted'],
    'accessed': dogs['accessed'],
    'days_since_posted': dogs['days_since_posted'].apply(lambda x: 0 if x < 0 else x)
}).set_index('id')

dogs_dates_days

### 11. Partition the dogs according to the number of weeks from the `posted` day to the day of last access.

Basically, we can reuse the same code from previous task, taking into accounting the 'weeks_since_posted' rather than the 'days_since_posted':

In [ ]:
dogs['weeks_since_posted'] = (dogs['days_since_posted'] // 7)

dogs_dates_weeks = pd.DataFrame({
    'id': dogs['id'].keys(),
    'posted': dogs['posted'],
    'accessed': dogs['accessed'],
    'weeks_since_posted': dogs['weeks_since_posted'].apply(lambda x: 0 if x < 0 else x)
}).set_index('id')

dogs_dates_weeks

### 12. Find for duplicates in the `dogs` dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

First of all, we must check if any dogs['description'] are empty: 

In [ ]:
dogs['description'].any()

Then we fill the empty rows in the 'dogs' dataset:

In [ ]:
dogs['description'].fillna('', inplace=True)

Now we can start performing data-related checks. Let's check if there are any rows in the dataset which share all the informations:

In [ ]:
FullDuplicateRows = dogs.duplicated()

FullDuplicateRows.any()

Since there are not duplicate rows which share all data, we can now perform another check before performing task number 12:

In [ ]:
DuplicateRows = dogs.duplicated(subset=['sex','breed_primary','breed_secondary', 'breed_mixed', 'breed_unknown', 'description'])

DuplicateRows.any()

Let's check how many rows this dataset has:

In [ ]:
DuplicateRows.sum()

Since there are rows which share sex, all the breeds and the entire description, we can remove them from the 'dogs' dataset; to do this, we will use store these rows in another dataset:

In [ ]:
cleaned_dogs = dogs.drop_duplicates(subset=['sex','breed_primary','breed_secondary', 'breed_mixed', 'breed_unknown', 'description'], keep='first', inplace=False, ignore_index=False)

After these steps, let's perform task 12:

In [ ]:
# Clean the description column by removing punctuation and converting to lowercase
cleaned_dogs['clean_description'] = cleaned_dogs['description'].str.replace('[^\w\s]', '').str.lower()

# Sort the dataframe by the 'clean_description' column
cleaned_dogs.sort_values('clean_description', inplace=True, ignore_index=False)

# Define a function to calculate the similarity between two strings
def compute_similarity(input_string, reference_string):
    if pd.isnull(input_string) or pd.isnull(reference_string) or input_string == "" or reference_string == "":
        return 0
    diff = difflib.ndiff(list(input_string), list(reference_string))
    diff_count = sum(1 for line in diff if line.startswith("-"))
    return 1 - (diff_count / len(input_string))

# Initialize an empty list to store the duplicate pairs
duplicate_pairs = []

# Iterate over the dataframe to compare adjacent rows
for i in range(len(cleaned_dogs) - 1):
    current_row = cleaned_dogs.iloc[i]
    next_row = cleaned_dogs.iloc[i + 1]
    
    if (current_row['breed_primary'] == next_row['breed_primary'] and
        current_row['breed_secondary'] == next_row['breed_secondary'] and
        current_row['breed_mixed'] == next_row['breed_mixed'] and
        current_row['breed_unknown'] == next_row['breed_unknown'] and
        current_row['sex'] == next_row['sex']):
        
        similarity = compute_similarity(current_row['clean_description'], next_row['clean_description'])
        if similarity >= 0.9:  # Adjust the similarity threshold as needed
            duplicate_pairs.append((current_row['id'], next_row['id']))

# Create a DataFrame to store the duplicate pairs
duplicate_pairs_df = pd.DataFrame(duplicate_pairs, columns=['Duplicate ID 1', 'Duplicate ID 2'])

# Print the duplicate pairs DataFrame
print(duplicate_pairs_df)

In [ ]:
duplicate_rows = pd.merge(duplicate_pairs_df, cleaned_dogs, left_on='Duplicate ID 1', right_on='id', suffixes=('', '_dup1'))
duplicate_rows = pd.merge(duplicate_rows, cleaned_dogs, left_on='Duplicate ID 2', right_on='id', suffixes=('_dup1', '_dup2'))

columns_to_view = ['sex_dup1', 'breed_primary_dup1', 'breed_secondary_dup1', 'breed_mixed_dup1', 'breed_unknown_dup1', 'clean_description_dup1',
                   'sex_dup2', 'breed_primary_dup2', 'breed_secondary_dup2', 'breed_mixed_dup2', 'breed_unknown_dup2', 'clean_description_dup2']

duplicate_rows = duplicate_rows[columns_to_view].rename(columns={
    'sex_dup1': 'Sex Duplicate 1',
    'breed_primary_dup1': 'Breed Primary Duplicate 1',
    'breed_secondary_dup1': 'Breed Secondary Duplicate 1',
    'breed_mixed_dup1': 'Breed Mixed Duplicate 1',
    'breed_unknown_dup1': 'Breed Unknown Duplicate 1',
    'clean_description_dup1': 'Clean Description Duplicate 1',
    'sex_dup2': 'Sex Duplicate 2',
    'breed_primary_dup2': 'Breed Primary Duplicate 2',
    'breed_secondary_dup2': 'Breed Secondary Duplicate 2',
    'breed_mixed_dup2': 'Breed Mixed Duplicate 2',
    'breed_unknown_dup2': 'Breed Unknown Duplicate 2',
    'clean_description_dup2': 'Clean Description Duplicate 2'
})

duplicate_rows